# Rozszerzenie relacji M2M

Czasami chcemy rozszerzyć informacje o jakimś powiazaniu. Może to być np. informacja o dacie złożenia zamówienia przez klienta (to będzie o2m, ponieważ klient może mieć wiele zamówień, jedno zamówienie może być przypisane tylko do jednego klienta) lub informacja o wysokości zakonktraktowanej gaży aktora za występ w filmie (to już będzie m2m). O ile w przypadku realcji o2o i o2m takie rozszerzenie poleg na dodaniu kolejnej kolumny w tabeli, w której umieściliśmy powiązanie, o tyle w przypadku relacji m2m dodatkową kolumnę umieszczamy w tabeli pośredniej. Tworząc relacje m2m w Django nie przejmowaliśmy się tabelą pośrednią. Wiemy, że Django wygeneruje taką tabelę automatycznie. Niestety, jeżeli chcemy rozszerzyć relacje m2m o dodatkową kolumnę to sami musimy już zdefniować tabelkę pośrednią i przypisać jej odpowiednie kolumny. W tym celu tworzymy tzw. model pośredni (ang. **intermediate model**)

W przykładzie (zaczerpniętym z dokumentacji Django) mamy dwie tabele - Person i Band.

<code>
class Person(models.Model):
    name = models.CharField(max_length=128)
</code>
<code>
    def __str__(self):
        return self.name
</code>


<code>
class Group(models.Model):
    name = models.CharField(max_length=128)
</code>
<code>
    def __str__(self):
        return self.name
</code>

Osoba może wchodzić w skład wielu zespołów, a zespół może składać się z wielu osób. Dodajemy relacji m2m (po stronie zespołu).

<code>
class Person(models.Model):
    name = models.CharField(max_length=128)
</code>
<code>
    def __str__(self):
        return self.name
</code>


<code>
class Group(models.Model):
    name = models.CharField(max_length=128)
    members = models.ManyToManyField(Person)
</code>
<code>
    def __str__(self):
        return self.name
</code>

I teraz załóżmy, że dodatkowo chcemy przechować też informacje o dacie dołączenia osoby do zespołu. Taka informacja będzie dotyczyła powiązania (tzn. data dotyczyć będzie połączenia konkretnej osoby z konkretnym zespołem), a nie samej osoby lub samego zespołu. W tym celu defniujemy model pośredni, nazywamy go Membership i umieszczajamy w nim wszystkie potrzebne kolumny. Następnie za pomocą parametru `through`, w polu ManyToManyField wskazujemy, że to pole ManyToManyField realizowane jest poprzez model pośredni `Membership`.

### Pełny przykład

<code>
class Person(models.Model):
    name = models.CharField(max_length=128)
</code>
<code>
    def __str__(self):
        return self.name
</code>


<code>
class Group(models.Model):
    name = models.CharField(max_length=128)
    members = models.ManyToManyField(Person, , through='Membership')
</code>
<code>
    def __str__(self):
        return self.name
</code>

<code>
class Membership(models.Model):
    person = models.ForeignKey(Person, on_delete=models.CASCADE)
    group = models.ForeignKey(Group, on_delete=models.CASCADE)
    date_joined = models.DateField()
</code>

Jeżeli nie masz powyższego przykładu w swoim projekcie, przeklej go (do pliku models.py wybranej aplikacji), a następnie przygotuj i wykonaj migracje. W dalszej części notatnika będziemy korzystać ze wszystkich trzech modeli.

Standradowo, najpierw linijki, które pozwolą nam swobodnie korzystać z Django w notatniku Jupyter.

In [ ]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'intro.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

Importujemy potrzebne biblioteki i modele

In [ ]:
from datetime import date

from orm_app.models import Person, Band, Membership

**C z CRUD**

In [ ]:
# Tworzymy wpisy Ringo Starr, John Lennon i Paul McCartney w tabeli Person i wpis The Beatles w tabeli Band
ringo = Person.objects.create(name="Ringo Starr")
john = Person.objects.create(name="John Lennon")
paul = Person.objects.create(name="Paul McCartney")

beatles = Band.objects.create(name="The Beatles")

Popatrzmy na utwrzone obiekty

In [ ]:
print(dir(beatles))  # widzimy menadżer powiązany - members

In [ ]:
print(dir(paul))  # widzimy menadżer powiązany - band_set

W obu modelach poza poznanym już wcześniej menadżerem powiązanym mamy też menadżer powiązany do modelu pośredniego - membership_set

Dodajemy powiązanie z zespołem

In [ ]:
# Version I (bezpośrednie użycie modelu pośredniego)
m1 = Membership(person=ringo, band=beatles, date_joined=date(1962, 8, 16))
m1.save()

In [ ]:
# Version II (użycie menadżera powiązanego modelu Band)
beatles.members.add(john, through_defaults={'date_joined': date(1960, 8, 1)})

# zwróć uwagę, że dane dla kolumn rozszerzających podajemy w postaci słownika przy użyciu parametru through_defaults, 
# w słowniku muszą znaleźć się wszystkie rozszerzające model pośredni dane.

In [ ]:
# Version III (użycie menadżera powiązanego modelu Person)
paul.band_set.add(beatles, through_defaults={'date_joined': date(1960, 8, 1)})

In [ ]:
# Version IV (za pomocą metody create - jednoczesne utworzenie wpisu i dodanie powiązania)
george = beatles.members.create(name="George Harrison", through_defaults={'date_joined': date(1960, 8, 1)})

# tutaj w parametrach trzeba podać wszystkie dane potrzebne do utworzenia wpisu w tabeli Person

In [ ]:
# Version V (za pomocą metody set - analogicznie jak w przypadk 'czystej' relacji m2m)
beatles.members.set([john, paul, ringo], through_defaults={'date_joined': date(1960, 8, 1)})

# metody set możemy użyć jeżeli wszystkie podane w liście wpisy mają te same wartości w kolumnach rozszerzających 
# model pośredni (czylie te same wartości w słowniku przypisywanym do parametru through_defaults).

**R z CRUD**

Znajdź wszystkie zespoły, które w składzie posiadają osoby z wartością w kolumnie name zaczynającą się na 'Paul'

In [ ]:
Band.objects.filter(members__name__startswith='Paul')

Znajdź wszystkie osoby, które przyłączyły się do zespołu 'The Beatles' po 1 Jan 1961

In [ ]:
Person.objects.filter(band__name='The Beatles', membership__date_joined__gt=date(1961, 1, 1))

A jak zapytać się o informacje uzupełniającą (w naszym przypadku o datę przyłączenia się do zespołu)? W którym roku do zespołu 'The Beatles' przyłączył się Ringo Starr?

In [ ]:
# Version I (przez bezpośrednie odpytanie modelu pośredniego)
ringos_membership = Membership.objects.get(band=beatles, person=ringo)
ringos_membership.date_joined

In [ ]:
# Version II (za pomocą menadżera powiązanego z modelem pośrednim z poziomu modelu Person)
ringos_membership = ringo.membership_set.get(band=beatles)
ringos_membership.date_joined

In [ ]:
# Version III (za pomocą menadżera powiązanego z modelem pośrednim z poziomu modelu Band)
ringos_membership = beatles.membership_set.get(person=ringo)
ringos_membership.date_joined

**U z CRUD**

**D z CRUD**

W jaki sposób możemy usuwać powiązania?

In [ ]:
# Version I (bezpośrednio z modelu pośredniego)
ringos_membership = Membership.objects.get(band=beatles, person=ringo)
ringos_membership.delete()

In [ ]:
# Version II (za pomocą metody remove menadżera powiązanego modelu Band)
beatles.members.remove(john)

# metoda remove zwraca None

In [ ]:
# Version III (za pomocą metody remove menadżera powiązanego modelu Person)
paul.band_set.remove(beatles)

**Q-objects** pozwalają nam na:
- składania operacji logicznych (operatory `or`, `and` i `not`)
- enkapsulację złożonych lookupów

Koniunkcję warunków (`and`) w metodach takich jak `filter` możemy uzyskać poprzez wypisanie poszczególnych warunków po przecinku.

In [ ]:
# Koniunkcja (and)
result = Customer.objects.filter(name__endswith='2', age__gte=40)
result

Z dokładnością do tego, że nie możemy dwa razy użyć parametru o tej samej nazwie.

In [ ]:
# nie można dwa razy użyć tego samego warunku
result = Customer.objects.filter(name__endswith='2', name__endswith='3')
result

Alternatywę warunków (`or`) możemy uzyskać poprzez użycie operatora `|` na rozdzielnych querysetach.

In [ ]:
# Alternatywa (or)
result = Customer.objects.filter(name__endswith='2') | Customer.objects.filter(age__gte=40)
result

Zaprzeczenia warunków w bezpośredni sposób nie otrzymamy. Możemy kombinować z użyciem metod przeciwnych, na przykład zamiast metody `filter` możemy użyć metody `exclude`, ale bezpośredniej metody nie ma. Przynajmniej dopóki nie użyjemy obiektu Q.

Obiekt Q służy do enkapsulacji pojedynczego lookupu, ew. grupy lookupów.

In [ ]:
from django.db.models import Q

In [ ]:
Q(name__endswith='2')

Następnie taki pojedynczy lookup, zamknięty w obiekcie Q może być składany z innym obiektem Q za pomocą operatorów `&` (and), `|` (or) ew. `^` (not).

In [ ]:
Q(name__endswith='2') & Q(age__gte=40)

In [ ]:
Q(age__gte=20) & Q(age__gte=40)

In [ ]:
~(Q(name__endswith='2') & Q(age__gte=40))

i nie ma ograniczenia na powtórzającą się nazwę parametru

In [ ]:
Q(name__endswith='2') | Q(age__gte=40)

Otrzymane złożenie możemy wykorzystać w dowolnej z metod manadżera, która wykorzystuje lookupy.

In [ ]:
# Koniunkcja (and) z użyciem obiektów Q - operator &
result = Customer.objects.filter(Q(name__endswith='2') & Q(age__gte=40))
result

In [ ]:
# Alternatywa (or) z użyciem obiektów Q - operator |
result = Customer.objects.filter(Q(name__endswith='2') | Q(age__gte=40))
result

In [ ]:
# Zaprzeczenie (not) z użyciem obiektów Q - operator ~
result = Customer.objects.filter(~(Q(name__endswith='2') | Q(age__gte=40)))
result

Enkapsulacja lookupów pozwala nam na tworzenie złożonych, ale wciąż czytelnych zapytań.

In [ ]:
lt_15_or_gt_30 = Q(age__lt=15) | Q(age__gt=30)
endswith_3 = Q(name__endswith='3')

Customer.objects.exclude(lt_15_or_gt_30 & endswith_3)